In [ ]:
import numpy as np
import math
import scipy.linalg as la
import matplotlib.pyplot as plt

In [ ]:
def RK34step(f, told, Yold, h):
    Y1 = f(told, Yold)
    Y2 = f(told + h/2, Yold + h/2*Y1)
    Y3 = f(told + h/2, Yold + h/2*Y2)
    Z3 = f(told + h, Yold - h*Y1 + 2*h*Y2)
    Y4 = f(told + h, Yold + h*Y3)
    Ynew = Yold + h/6*(Y1 + 2*Y2 + 2*Y3 + Y4)

    err = np.abs(h/6*(2*Y2+Z3-2*Y3-Y4))
    return Ynew, err

def newstep (tol, err, errold, hold, k):
    hnew = hold * np.power(tol/max(err), 2/(3*k))*np.power(tol/max(errold), -1/(3*k))
    return hnew
    
def adaptiveRK34(f, t0, tf, y0, tol):
    k = 4
    h = (tf-t0)*np.power(tol,1/4)/(1000*(1+la.norm(f(t0,y0))))
    t = [t0]
    Y = [y0]
    ERR = [np.zeros_like(y0)]
    y, err = RK34step(f, t[-1], y0, h)
    Y.append(y)
    ERR.append(err)
    t.append(t[-1] + h)
    h = newstep(tol, ERR[-1], np.array([tol, tol]), h, k)


    while t[-1] + h < tf:
        y, err = RK34step(f, t[-1], Y[-1], h)
        Y.append(y)
        ERR.append(err)
        t.append(t[-1] + h)
        h = newstep(tol, ERR[-1], ERR[-2], h, k)
    

    y, err = RK34step(f, t[-1], Y[-1], tf-t[-1])
    t.append(tf)
    Y.append(y)
    ERR.append(err)
    return t, Y, ERR


In [ ]:
def VanDerPol(t,u):
    return np.array([u[1], mu*(1-u[0]**2)*u[1]-u[0]])

### Task 3.1

In [ ]:
mu = 100
t, Y, ERR = adaptiveRK34(VanDerPol, 0, 2*mu, np.array([2,0]), 1e-6)
x = [Yx[0] for Yx in Y]
y = [Yy[1] for Yy in Y]
plt.grid()
plt.title("Y1 plotted versus time")
plt.plot(t, y)
plt.show()
plt.grid()
plt.title("Investigation of periodicity") # Ett problem är stiff om alla termerna är chill, I grafen nedan kan vi se ett typisk o chill beteende på två ställen 
plt.plot(x,y)
plt.show()

## Task 3.2

In [ ]:
mus = [10, 15, 22, 33, 47, 68, 100, 150, 220, 330, 470, 680, 1000] #Olika värden på my. 
steps = []
for mu in mus:
    t, Y, ERR = adaptiveRK34(VanDerPol, 0, 0.7*mu, np.array([2,0]), 1e-6)
    steps.append(len(t))
    x = [Yx[0] for Yx in Y]
    y = [Yy[1] for Yy in Y]
    plt.plot(t, y)
    plt.grid()
    plt.title("μ was equal to " + str(mu))
    plt.xlabel("Time")
    plt.ylabel("y2")
    plt.show()

In [ ]:
plt.grid()
plt.title("Steps needed for different values on μ")
print(steps)
plt.loglog(mus, steps)

## Task 3.3

In [ ]:
from scipy.integrate import solve_ivp
steps = []

for mu in mus:
    solution = solve_ivp(VanDerPol, [0, 0.7*mu], np.array([2,0]), atol=1e-6, method='BDF')  
    steps.append(len(solution.t))
    # plt.plot(solution.t, solution.y[1])
    # plt.title("μ was equal to " + str(mu))
    # plt.xlabel("Time")
    # plt.ylabel("y2")
    # plt.grid()
    # plt.show()

plt.grid()
plt.title("μ versus steps")
plt.xlabel("Tested μ")
plt.ylabel("Amount of steps needed")
plt.loglog(mus, steps)
plt.show()
print(steps)


In [ ]:
mu = 10000
solution = solve_ivp(VanDerPol, [0, 0.7*mu], np.array([2,0]), atol=1e-6, method='BDF')  ##Ändrade till atol = absolut tolerance
steps.append(len(solution.t))
plt.plot(solution.t, solution.y[1])
plt.title("μ was equal to " + str(mu))
plt.xlabel("Time")
plt.ylabel("y2")
plt.grid()
plt.show() #Aslätt, vi har ju bara problem på 2 ställen, och då kan denna metod minska h jättemycket där, och sedan öka den igen.